In [197]:
# -*- coding: utf-8 -*-

import functools
import numpy as np
import sklearn.metrics
import sklearn.datasets
import sklearn.model_selection


import pso
import ann

np.random.seed(6663)
def dim_weights(shape):
    dim = 0
    for i in range(len(shape)-1):
        dim = dim + (shape[i] + 1) * shape[i+1]
    return dim

def weights_to_vector(weights):
    w = np.asarray([])
    for i in range(len(weights)):
        v = weights[i].flatten()
        w = np.append(w, v)
    return w

def vector_to_weights(vector, shape):
    weights = []
    idx = 0
    for i in range(len(shape)-1):
        r = shape[i+1]
        c = shape[i] + 1
        idx_min = idx
        idx_max = idx + r*c
        W = vector[idx_min:idx_max].reshape(r,c)
        weights.append(W)
    return weights

def eval_neural_network(weights, shape, X, y):
    mse = np.asarray([])
    for w in weights:
        weights = vector_to_weights(w, shape)
        nn = ann.MultiLayerPerceptron(shape, weights=weights)
        y_pred = nn.run(X)
        mse = np.append(mse, sklearn.metrics.mean_squared_error(np.atleast_2d(y), y_pred))
    return mse

def print_best_particle(best_particle):
    print("New best particle found at iteration #{i} with mean squared error: {score}".format(i=best_particle[0], score=best_particle[1]))




#just xy error = 0.22938741634873416
#with sinx sin y error = 0.2325498216710004
#with sin and sq error = 0.21790465807135315

In [198]:


def sine(X):
    return np.sin(X)


#loading Starts here

def sq(X):
    return np.power(X,2)
num_classes = 2
f = open("two_spirals.dat","r")
data = np.loadtxt(f)

# X = data[:,0:2]+sine(data[:,0:2])+sq(data[:,0:2])
# #X = data[:,0:2]+sine(data[:,0:2])
# #X = data[:,0:2]
# y = data[:,2]
# y = y.astype(int)
# train_test_split = int(0.5*(len(X)))
# #print(train_test_split)
# X_train,X_test = X[:train_test_split],X[train_test_split:]
# y_train,y_test = y[:train_test_split],y[train_test_split:]

# print(len(X))
# print(y.size)
# print(y_train.size)
# print(y_test.size)

In [199]:
X = data[:,0:2]
test = np.zeros(shape=(len(X),6))

for i in range(len(X)):
   # print(X[i])
    #print(sine(X[i]))
    test[i] = np.append(X[i],[sine(X[i]),sq(X[i])])
X=test
y = data[:,2]
y = y.astype(int)
train_test_split = int(0.5*(len(y)))
#print(train_test_split)
X_train,X_test = X[:train_test_split],X[train_test_split:]
y_train,y_test = y[:train_test_split],y[train_test_split:]


print(len(test))
print(len(X))
print(y.size)
print(y_train.size)
print(y_test.size)
print(X[0])

264
264
264
132
132
[ 4.47485    -0.89004    -0.97192002 -0.77709692 20.02428252  0.7921712 ]


In [200]:
print(y_train.size)


132


In [205]:

# # Load MNIST digits from sklearn

# num_classes = 10
# mnist = sklearn.datasets.load_digits(num_classes)
# X, X_test, y, y_test = sklearn.model_selection.train_test_split(mnist.data, mnist.target)

num_inputs = X.shape[1]
print("Number of inputs = ",num_inputs)
y_true = np.zeros((len(y), num_classes))

for i in range(len(y)):
    y_true[i, y[i]] = 1

y_test_true = np.zeros((len(y_test), num_classes))
for i in range(len(y_test)):
    y_test_true[i, y_test[i]] = 1


print("y test shap e= ", y_test.shape)
print("y true shape = ",y_true.shape)
print("y test true shape = ",y_test_true.shape)
print(X_test.shape)
print(X_train.shape)

Number of inputs =  6
y test shap e=  (132,)
y true shape =  (264, 2)
y test true shape =  (132, 2)
(132, 6)
(132, 6)


In [210]:

# Set up

shape = (num_inputs, 8, num_classes)

cost_func = functools.partial(eval_neural_network, shape=shape, X=X, y=y_true.T)

swarm = pso.ParticleSwarm(cost_func, num_dimensions=dim_weights(shape), num_particles=30,chi=0.72984 ,phi_p=2.02,phi_g=2.02)

# Train...
i = 0
best_scores = [(i, swarm.best_score)]
print_best_particle(best_scores[-1])
while swarm.best_score>1e-6 and i<4000:
    swarm._update()
    i = i+1
    #print("index = ",i)
    if i%100==0 and swarm.best_score < best_scores[-1][1]:
        best_scores.append((i, swarm.best_score))
        print_best_particle(best_scores[-1])

# Test...
best_weights = vector_to_weights(swarm.g, shape)
best_nn = ann.MultiLayerPerceptron(shape, weights=best_weights)

y_test_pred = np.round(best_nn.run(X_test))
print("MSE score =",sklearn.metrics.mean_squared_error(y_test_true,y_test_pred.T))
print("Accuracy score = ",sklearn.metrics.accuracy_score(y_test_true,y_test_pred.T))

print(sklearn.metrics.classification_report(y_test_true, y_test_pred.T))

# #MES score = 0.30303030303030304
# Accuracy score =  0.5606060606060606


New best particle found at iteration #0 with mean squared error: 0.4037024898701077
New best particle found at iteration #100 with mean squared error: 0.21216305452025241
New best particle found at iteration #200 with mean squared error: 0.2006584772461695
New best particle found at iteration #300 with mean squared error: 0.18233487446101682
New best particle found at iteration #400 with mean squared error: 0.1746649276578314
New best particle found at iteration #500 with mean squared error: 0.16942572125038705
New best particle found at iteration #600 with mean squared error: 0.16047439925846213
New best particle found at iteration #700 with mean squared error: 0.15041963211983447
New best particle found at iteration #800 with mean squared error: 0.1442599846612074
New best particle found at iteration #900 with mean squared error: 0.13338956057853307
New best particle found at iteration #1000 with mean squared error: 0.11575332882042053
New best particle found at iteration #1100 with 

In [208]:
'''
Using the brute force method figure out the optimal functions

then use the same functions in PSO and compare it with base line

'''

'\nUsing the brute force method figure out the optimal functions\n\nthen use the same functions in PSO and compare it with base line\n\n'